In [1]:
import pandas as pd
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import bokeh as bk
from bokeh.plotting import figure, show
import datetime
import sqlite3

In [3]:
with open('signal_names.csv','r') as f:
    signal_list = [line.strip().split(',') for line in f.readlines()]
    

signals_df = pd.DataFrame([line[:2] + sorted(line[2:]) for line in signal_list[1:]], columns = signal_list[0])
signals_df['intersection'] = signals_df['cross_street_1'] + ' & ' + signals_df['cross_street_2']

In [3]:
signals_df = signals_df[signals_df['cross_street_1'] != 'no_int_found']

In [4]:
signals_df = signals_df[['intersection', 'lat', 'lon']].drop_duplicates('intersection').reset_index().drop('index',axis=1)
signals_df.head()

,intersection,lat,lon
0,Central Park W & W 106th St,40.7980486,-73.9600434
1,Manhattan Ave & W 106th St,40.798645,-73.9614743
2,Columbus Ave & W 106th St,40.7992545,-73.962886
3,Amsterdam Ave & W 106th St,40.8004313,-73.9657046
4,Broadway & W 106th St,40.8014007,-73.9679942


In [12]:
from shapely.geometry import Point
from shapely.geometry import MultiLineString
from shapely.wkt import loads


signals_df['coord'] = list(zip(*(signals_df['lon'].astype(float),signals_df['lat'].astype(float))))
signals_df['loc'] = signals_df['coord'].map(lambda x: Point(x))

In [10]:
hump_df = pd.read_csv('dot_VZV_Speed_Humps_20190604.csv')
hump_df['date'] = hump_df['date_insta'].map(lambda x: datetime.datetime.strptime(x[:10],'%m/%d/%Y'))
#hump_df.groupby('date').count()

In [13]:
hump_df['lines'] = hump_df['the_geom'].map(lambda x: loads(x))

In [16]:
%load_ext sql
%sql sqlite:///safe_cycle_db.sqlite

'Connected: @safe_cycle_db.sqlite'

In [5]:
int_list = list(zip(signals_df.index, signals_df['intersection'], signals_df['lat'].astype(float), signals_df['lon'].astype(float)))

In [8]:
%%sql 

DROP TABLE IF EXISTS intersections;

CREATE TABLE intersections (
    id INTEGER PRIMARY KEY NOT NULL, 
    name MEDIUMTEXT, 
    lat FLOAT, 
    lon FLOAT);

 * sqlite:///safe_cycle_db.sqlite
Done.
Done.


[]

In [17]:
%%sql

SELECT * FROM intersections

 * sqlite:///safe_cycle_db.sqlite
Done.


id,name,lat,lon
0,Central Park W & W 106th St,40.7980486,-73.9600434
1,Manhattan Ave & W 106th St,40.798645,-73.9614743
2,Columbus Ave & W 106th St,40.7992545,-73.962886
3,Amsterdam Ave & W 106th St,40.8004313,-73.9657046
4,Broadway & W 106th St,40.8014007,-73.9679942
5,Duke Ellington Blvd & W End Ave,40.8016434,-73.9685615
6,Columbus Ave & W 80th St,40.7826312,-73.9749879
7,Amsterdam Ave & W 80th St,40.783828,-73.9778254
8,Broadway & W 80th St,40.7845691,-73.9795827
9,W 80th St & W End Ave,40.7850311,-73.9806779


In [6]:
conn = sqlite3.connect('safe_cycle_db.sqlite')
cur = conn.cursor()

In [7]:
cur.execute('''DROP TABLE IF EXISTS intersections;''')

cur.execute('''CREATE TABLE intersections (
    id INTEGER PRIMARY KEY NOT NULL, 
    name MEDIUMTEXT, 
    lat FLOAT, 
    lon FLOAT);''')
conn.commit()

In [8]:
for tup in int_list:
    string = '''INSERT INTO intersections (id, name, lat, lon) 
                VALUES ({}, "{}",{},{})'''.format(tup[0],tup[1],tup[2],tup[3])
    #print(string)
    cur.execute(string)
    
conn.commit()
print(string)

INSERT INTO intersections (id, name, lat, lon) 
                VALUES (9737, "Clarendon Rd & Nostrand Ave",40.6432433,-73.9487606)


In [14]:
print(string)

INSERT INTO intersections (id, name, lat, lon) 
                VALUES (3312, 'Albany Ave & St Mark's Ave',40.674599,-73.938895)


In [9]:
%load_ext sql
%sql sqlite:///safe_cycle_db.sqlite

'Connected: @safe_cycle_db.sqlite'

In [10]:
%%sql

SELECT * FROM intersections LIMIT 100

 * sqlite:///safe_cycle_db.sqlite
Done.


id,name,lat,lon
0,Central Park W & W 106th St,40.7980486,-73.9600434
1,Manhattan Ave & W 106th St,40.798645,-73.9614743
2,Columbus Ave & W 106th St,40.7992545,-73.962886
3,Amsterdam Ave & W 106th St,40.8004313,-73.9657046
4,Broadway & W 106th St,40.8014007,-73.9679942
5,Duke Ellington Blvd & W End Ave,40.8016434,-73.9685615
6,Columbus Ave & W 80th St,40.7826312,-73.9749879
7,Amsterdam Ave & W 80th St,40.783828,-73.9778254
8,Broadway & W 80th St,40.7845691,-73.9795827
9,W 80th St & W End Ave,40.7850311,-73.9806779


In [ ]:
cur.execute('''DROP TABLE IF EXISTS crashes;''')

cur.execute('''CREATE TABLE intersections (
    id INTEGER PRIMARY KEY NOT NULL, 
    name MEDIUMTEXT, 
    lat FLOAT, 
    lon FLOAT);''')
conn.commit()

In [240]:
for i, el in enumerate(int_list):
    int_list[i] = (el[0], el[1].replace('and','&'),el[2], el[3])
int_list

[(0, 'Central Park W & W 106th St', 40.7980486, -73.9600434),
 (1, 'Manhattan Ave & W 106th St', 40.798645, -73.9614743),
 (2, 'Columbus Ave & W 106th St', 40.7992545, -73.962886),
 (3, 'Amsterdam Ave & W 106th St', 40.8004313, -73.9657046),
 (4, 'Broadway & W 106th St', 40.8014007, -73.9679942),
 (5, 'Duke Ellington Blvd & W End Ave', 40.8016434, -73.9685615),
 (6, 'Columbus Ave & W 80th St', 40.7826312, -73.9749879),
 (7, 'Amsterdam Ave & W 80th St', 40.783828, -73.9778254),
 (8, 'Broadway & W 80th St', 40.7845691, -73.9795827),
 (9, 'W 80th St & W End Ave', 40.7850311, -73.9806779),
 (10, 'Broadway & Hillside Ave', 40.8593027, -73.9313326),
 (11, 'Central Park W & W 84th St', 40.7839788, -73.9703286),
 (12, 'Columbus Ave & W 84th St', 40.7851655, -73.9731396),
 (13, 'Amsterdam Ave & W 84th St', 40.7863627, -73.9759753),
 (14, 'Broadway & W 84th St', 40.7870877, -73.9776927),
 (15, 'W 84th St & W End Ave', 40.7875673, -73.9788289),
 (16, 'Edgar Allen Poe St & Riverside Dr', 40.788513

In [188]:
for tup in int_list:
    %sql INSERT INTO intersections VALUES tup

 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO inte

(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://s

(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://s

(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://s

   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background

...

(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://s

(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://s

   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background

 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO inte

(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///safe_cycle_db.sqlite
   sqlite:///testdb.sqlite
(sqlite3.OperationalError) near "tup": syntax error [SQL: 'INSERT INTO intersections VALUES tup'] (Background on this error at: http://s

In [61]:
hump_df.head()


,on_st,the_geom,to_st,from_st,new_humps,date_insta,Shape_Leng,date,lines
0,1 STREET,MULTILINESTRING ((-73.9799168591299 40.6728580...,7 AVENUE,6 AVENUE,2,05/22/2014 12:00:00 AM +0000,775.659477,2014-05-22,(LINESTRING (-73.9799168591299 40.672858023374...
1,1 STREET,MULTILINESTRING ((-73.99294738655144 40.678995...,BOND STREET,HOYT STREET,2,09/11/2017 12:00:00 AM +0000,652.150192,2017-09-11,(LINESTRING (-73.99294738655144 40.67899544181...
2,100 AVENUE,MULTILINESTRING ((-73.73560306475389 40.714945...,SPRINGFIELD BL.,100 DR./220 STREET,1,12/15/2011 12:00:00 AM +0000,423.930065,2011-12-15,(LINESTRING (-73.73560306475389 40.71494534147...
3,100 ST,MULTILINESTRING ((-74.03364741744534 40.612833...,FT HAMILTON PKWY,FOURTH AVE,1,08/01/1996 12:00:00 AM +0000,591.886182,1996-08-01,(LINESTRING (-74.03364741744534 40.61283324315...
4,100 Street,MULTILINESTRING ((-73.86848374851515 40.757334...,34TH Avenue,Northern Boulevard,1,06/17/2004 12:00:00 AM +0000,673.502404,2004-06-17,(LINESTRING (-73.86848374851515 40.75733493687...


In [121]:
def get_bumps(hump_df, int_df):
    
    int_df['speed_bumps'] = list(zip(np.zeros(int_df['coord'].shape),np.zeros(int_df['coord'].shape)))
    
    for ix in hump_df.index:
        multi = hump_df.loc[ix]['lines']
        num_bumps = hump_df.loc[ix]['new_humps']
        date = hump_df.loc[ix]['date']
        locs = int_df[int_df['loc'].map(lambda x: multi.distance(x) < .0001)].index
        int_df.at[locs,'speed_bumps'] = int_df.loc[locs]['speed_bumps'].map(lambda x: (x[0] + num_bumps, date) )
        
    return int_df

In [122]:
hump_df.drop(hump_df[hump_df['the_geom'].str.contains('EMPTY')].index,inplace=True)
hump_df[hump_df['the_geom'].str.contains('EMPTY')]

,on_st,the_geom,to_st,from_st,new_humps,date_insta,Shape_Leng,date,lines


In [125]:
signals_df = get_bumps(hump_df,signals_df)
len(signals_df[signals_df['speed_bumps'] != (0.0,0.0)])

1569

In [132]:
def get_line_feature(feature_df, int_df, feat_type):
    
    int_df[feat_type] = list(zip(np.zeros(int_df['coord'].shape),np.zeros(int_df['coord'].shape)))
    
    feature_df['lines'] = feature_df['the_geom'].map(lambda x: loads(x))
    
    for ix in feature_df.index:
        multi = feature_df.loc[ix]['lines']
        if feat_type in []:
            num_feat = hump_df.loc[ix]['']
        else:
            num_feat = 1
        date = feature_df.loc[ix]['ENFORCEMEN']
        locs = int_df[int_df['loc'].map(lambda x: multi.distance(x) < .0001)].index
        int_df.at[locs,feat_type] = int_df.loc[locs][feat_type].map(lambda x: (x[0] + num_feat, date) )
        
    return int_df

In [133]:
art_slowzone_df = pd.read_csv('dot_VZV_Arterial_Slow_Zones_20190604.csv')
art_slowzone_df.head()

,FROM_STREE,ENFORCEMEN,TO_STREET,the_geom,Shape_Leng
0,Columbia St,05/19/2014 12:00:00 AM +0000,76 St,MULTILINESTRING ((-73.89867564098704 40.675582...,80344.034362
1,E 140th St,05/19/2014 12:00:00 AM +0000,Mosholu Parkway,MULTILINESTRING ((-73.92988099693609 40.813040...,76627.888146
2,40 Rd,06/02/2014 12:00:00 AM +0000,114 St,MULTILINESTRING ((-73.93474259188893 40.750979...,22288.089025
3,Freeman St,06/02/2014 12:00:00 AM +0000,Bayard St,MULTILINESTRING ((-73.94499614744701 40.720773...,11108.542243
4,Jerome Ave,06/09/2014 12:00:00 AM +0000,NE Thrwy SB,MULTILINESTRING ((-73.881244225275 40.88270963...,15702.229781


In [139]:
signals_df = get_line_feature(art_slowzone_df,signals_df, 'art_slowzone')

In [140]:
len(signals_df[signals_df['art_slowzone'] != (0.0,0.0)])

1314

In [143]:
signals_df.to_pickle('signals_df_speedbump_artSZ.pkl')

In [156]:
def get_line_feature_3(feature_df, int_df, feat_type):
    
    int_df[feat_type] = list(zip(np.zeros(int_df['coord'].shape),np.zeros(int_df['coord'].shape)))
    
    feature_df['lines'] = feature_df['the_geom'].map(lambda x: loads(x))
    
    for ix in feature_df.index:
        multi = feature_df.loc[ix]['lines']
        if feat_type in []:
            num_feat = hump_df.loc[ix]['']
        else:
            num_feat = 1
        date = feature_df.loc[ix]['date']
        locs = int_df[int_df['loc'].map(lambda x: multi.distance(x) < .0001)].index
        int_df.at[locs,feat_type] = int_df.loc[locs][feat_type].map(lambda x: (x[0] + num_feat, date) )
        
    return int_df

In [153]:
signal_timing_df = pd.read_csv('dot_VZV_Signal_Timing_20190604.csv')
print(len(signal_timing_df))
signal_timing_df['date'] = signal_timing_df['YR'].map(lambda x: datetime.datetime.strptime(str(x),'%Y'))

206


In [157]:
signals_df = get_line_feature_3(signal_timing_df,signals_df,'signal_timing')

In [159]:
len(signals_df[signals_df['signal_timing']!= (0.0,0.0)])

3703

In [160]:
signals_df.head()

,intersection,lat,lon,coord,loc,speed_bumps,art_slowzone,signal_timing
0,Central Park W & W 106th St,40.7980486,-73.9600434,"(-73.9600434, 40.7980486)",POINT (-73.9600434 40.7980486),"(0.0, 0.0)","(0.0, 0.0)","(1.0, 2018-01-01 00:00:00)"
1,Manhattan Ave & W 106th St,40.798645,-73.9614743,"(-73.9614743, 40.798645)",POINT (-73.96147430000001 40.798645),"(0.0, 0.0)","(0.0, 0.0)","(0.0, 0.0)"
2,Columbus Ave & W 106th St,40.7992545,-73.962886,"(-73.962886, 40.7992545)",POINT (-73.962886 40.7992545),"(0.0, 0.0)","(0.0, 0.0)","(0.0, 0.0)"
3,Amsterdam Ave & W 106th St,40.8004313,-73.9657046,"(-73.9657046, 40.8004313)",POINT (-73.9657046 40.8004313),"(0.0, 0.0)","(1.0, 11/17/2014 12:00:00 AM +0000)","(2.0, 2017-01-01 00:00:00)"
4,Broadway & W 106th St,40.8014007,-73.9679942,"(-73.9679942, 40.8014007)",POINT (-73.96799420000001 40.8014007),"(0.0, 0.0)","(1.0, 08/04/2014 12:00:00 AM +0000)","(0.0, 0.0)"


In [166]:
for s in signal_timing_df['OnStreet'].unique():
    if s[:2] == '10':
        print(s)

10 Avenue
104 ST


In [167]:
signals_df.to_pickle('signals_df_speedbump_artSZ_sigtiming.pkl')

In [67]:
import matplotlib.path as mplPath
import numpy as np

poly = [190, 50, 500, 310]
bbPath = mplPath.Path(np.array([[poly[0], poly[1]],
                     [poly[1], poly[2]],
                     [poly[2], poly[3]],
                     [poly[3], poly[0]]]))

bbPath.contains_point((200, 100))

True